# Objetivo de este notebook

Se exhibiran algunos conceptos, algoritmos o funciones para establecer un modelo de recomendación de películas.

In [1]:
#Para correr las funciones de la librería libmf de python de este notebook 
#se debe clonar el repo de https://github.com/PorkShoulderHolder/python-libmf.git
#y subir este notebook dentro de la carpeta /libmf/tests/

#Para correr las funciones de libmf de este notebook ejecutados con 
#Shell Commands ("!") y Shell-Related Magic Commands ("%")
#se debe clonar el repo: https://github.com/cjlin1/libmf.git

#El clonar estos archivos es necesario porque contienen funciones programadas o
#ejemplos construidos para exhibir el uso de estos paquetes.

#Existen bases que se requieren tener la carpeta donde se corra el código
#a saber: movies_title_fix.csv dat_muestra_nflix.csv

In [2]:
#Si se queire este notebook a través de docker con la imagen de
#jupyer numerical se deben instalar los siguientes paquetes:
#pip install libmf
#pip install colorama

In [3]:
import sys
import numpy as np
import types
import scipy
from scipy.sparse import random
from scipy import stats
import pandas as pd
import copy
import random
from functools import partial
from colorama import init

import libmf
from libmf import mf

Using file found in /home/miuser/.local/lib/python3.6/site-packages:
/home/miuser/.local/lib/python3.6/site-packages/libmf.cpython-36m-x86_64-linux-gnu.so


# Funciones auxiliares
Para explicar y ejemplificar el uso de libmf

In [4]:
def constante(x,useless):
    return x

In [5]:
def perform(fun, *args):
    return fun(*args)

In [6]:
def nan_or_value(x, f, *args):
    if(x==1):
        return perform(f,*args)
    else:
        return np.nan

In [7]:
nan_or_value_vec=np.vectorize(nan_or_value, otypes=[object])

In [8]:
def crear_matriz_rala(n_ren,m_col,prop_elem_difer_nan,func_rand,*args_func_rand):
    tot=int(n_ren*m_col)
    k=int(tot*prop_elem_difer_nan)
    M=np.concatenate((np.zeros(tot-k).astype(int), np.ones(k).astype(int)))
    M=np.array(random.sample(list(M),tot))
    M=M.reshape(n_ren,m_col)
    M=nan_or_value_vec(M,func_rand,*args_func_rand).astype(float)
    return M

In [9]:
def crear_matriz_rala_compacta(n_ren, m_col, prop_elem_difer_nan):
    tot=int(n_ren*m_col)
    k=int(tot*prop_elem_difer_nan)
    M=random.sample(range(tot), k)
    columnas=np.mod(M,m_col)
    renglones=np.divide(M,m_col).astype(int)
    aleatorios=np.random.randint(1, 6, k)
    M=np.concatenate([renglones, columnas, aleatorios])
    M=M.reshape(3,k).T
    return M

In [10]:
def compactar_matriz_rala(M):
    if type(M) is not np.ndarray:
        sys.exit('M debe ser de tipo numpy.ndarray')
    elif np.isnan(M).sum()==len(M):
        sys.exit('Todos los elementos de M son nan')
    MC=np.zeros((M[np.isnan(M)==False].size,3))
    k = 0
    n,m=M.shape
    for i in range(n): 
        for j in range(m): 
            if (~np.isnan(M[i,j])): 
                MC[k,0] = i
                MC[k,1] = j
                MC[k,2] = M[i,j] 
                k += 1
    return MC

In [11]:
#Elegiremos un formato general para visualizar las matrices exhibidas como ejemplo
np.set_printoptions(sign=' ', precision=3)
#np.set_printoptions(sign=' ',formatter={'float': lambda x: "{0:0.3f}".format(x)})

In [12]:
def modelo_base_ref(MC):
    df=pd.DataFrame(MC,columns=['row','col','score'])
    #Media total
    c=df['score'].mean(axis=0)

    #Media por columna
    mean_by_col=df[['col','score']].groupby(['col']).mean()
    mean_by_col.rename(columns={'score':'mean_score_by_col'}, inplace=True)
    #Media por renglón
    mean_by_row=df[['row','score']].groupby(['row']).mean()
    mean_by_row.rename(columns={'score':'mean_score_by_row'}, inplace=True)

    #Se unen todas los resultados obtenidos, incluyendo la columna de
    #índice de columnas y renglones
    df=df.set_index('col').join(mean_by_col,how='left')
    df['col']=df.index
    df=df.set_index('row').join(mean_by_row,how='left')
    df['row']=df.index
    df=df[['row','col','mean_score_by_row','mean_score_by_col']]
    df['total_score_mean']=c
    
    return df

In [13]:
#Esta función filtra los 'índices de renglones (usuarios)' o
#'índices de columna (películas)' cuyo conteo total es mayor a n
def filtrar(MC,ren_o_col,n):
    MC_df=pd.DataFrame(MC,columns=['row','col','score'])
    
    #Este pequeño bloque calcula el total de elementos por columna o renglón
    count_by=MC_df[[ren_o_col,'score']].groupby([ren_o_col]).count()
    count_by.rename(columns={'score':'tot_counted'}, inplace=True)
    count_by[ren_o_col]=count_by.index
    count_by=count_by[[ren_o_col,'tot_counted']]
    
    #En este bloque se filtran los elementos de la matriz cuyo conteo por
    #columna o renglón tienen más de n elementos
    count_by=count_by[count_by.apply(lambda x: x['tot_counted'] > n, axis=1)]
    count_by=MC_df.set_index(ren_o_col).join(count_by.set_index(ren_o_col),how='inner')
    count_by[ren_o_col]=count_by.index
    count_by=count_by[['row','col','score']]
    
    return count_by.to_numpy()

In [71]:
def muestra_val_ent(df_netflix,prop):
    #Base de todos los usuarios
    usuarios=pd.DataFrame(df_netflix['usuario_id'].unique(), columns=['usuario_id'])
    #Muestreo de usuarios
    valida_usuarios=usuarios.sample(frac=prop)  

    #Base de todas las películas
    peliculas=pd.DataFrame(df_netflix['peli_id'].unique(), columns=['peli_id'])
    #Muestreo de películas
    valida_pelis=peliculas.sample(frac=0.2)
    
    #df_valida tiene las columnas usuario_id, peli_id y calif, pero
    #sólo incluye los usuarios muestreados en valida_usuarios
    df_valida=pd.merge(df_netflix,valida_usuarios, on ='usuario_id', how='inner')
    #data_valida filtra de la base df_valida los pelis_id muestreas
    #en valida_pelis
    data_test=pd.merge(df_valida,valida_pelis, on ='peli_id', how='inner')
    left=pd.merge(df_netflix, data_test, how='left',on=['usuario_id', 'peli_id'])
    data_training=left[left.isnull().any(axis=1)][['usuario_id', 'peli_id','calif_x']]
    return data_test, data_training

# Matrices ralas (sparse)

In [14]:
#En este proyecto se exhibe el uso de las funciones de la librería libmf
#relacionadas a la factorización de matrices. El proyecto se enfoca al uso
#de estos algoritmos/métodos para los sistemas de recomendación de películas
#cuyo objetivo será obtener recomendaciones de películas para usuarios
#contenidos dentro de una base de datos proporcionada por netflix

#Si bien la base tiene datos como peli_id, usuario_id, calif, fecha, el
#algoritmo solo trabajará con los datos de calificaciones. Si bien existen
#diversos usuarios y películas, son pocas películas de las que se tienen
#calificaciones (escala del 1 al 5) proporcionadas por usuarios, por lo que
#la representación de estas calificaciones a través de una matriz (donde
#los usuarios serán los índices de los rengloens, las películas los índices
#de columnas y las calificaciones los valores de las entradas de la matriz),
#contendrá muchos huecos o calificaciones inexistentes,
#es decir, es una matriz rala (llena de ceros o valores no proporcionados)
#No obstante lo anterior, debido al numero de usuarios y películas
#existen muchos datos, y para no guardar todos los valores de cero (o NA)
#se opta por expresar y guardar dicha matriz en una forma compacta,
#la cual contiene por cada calificación, el usuario que la generó y la película
#a la que corrresponde (es decir, es una matriz con 3 columnas)

In [15]:
#Crear matriz rala de tamaño 20x5, donde el 25% de sus elementos
#será distinto de nan, señalando la distribución de probabilidad
#que tendrán dichos elementos
M=crear_matriz_rala(20,5,0.25,stats.randint.rvs,*[1,6])
#Se puede cambiar la distribución aleatoria. Probar con las
#distribuciones existentes en scipy.stats o la función constante
#R=crear_matriz_rala(20,5,0.5,scipy.stats.uniform.rvs,*[1,4])
#R=crear_matriz_rala(20,5,0.2,stats.binom.rvs,*[5,0.2])
#R=crear_matriz_rala(20,5,0.5,constante,*[1,0])
print('Matriz M:\n',M)
print('Tamaño M:\n',M.shape)
print('Numero de elementos diferentes de nan:\n',M[np.isnan(M)==False].size)

Matriz M:
 [[ nan  nan   2.  nan   4.]
 [ nan  nan  nan   4.   4.]
 [ nan   5.  nan  nan  nan]
 [ nan  nan   3.  nan  nan]
 [ nan  nan  nan  nan   4.]
 [ nan   4.  nan  nan   4.]
 [  4.  nan  nan  nan  nan]
 [ nan  nan  nan  nan  nan]
 [ nan  nan  nan   1.  nan]
 [ nan  nan  nan  nan  nan]
 [ nan  nan  nan  nan  nan]
 [ nan  nan   5.   1.  nan]
 [ nan   5.   1.  nan  nan]
 [  2.  nan  nan  nan  nan]
 [  1.  nan  nan  nan   1.]
 [ nan  nan  nan   3.   5.]
 [ nan   3.  nan  nan   3.]
 [ nan  nan  nan  nan  nan]
 [ nan   5.   4.  nan  nan]
 [  1.  nan  nan  nan  nan]]
Tamaño M:
 (20, 5)
Numero de elementos diferentes de nan:
 25


In [16]:
#Expresión compacta de la matriz rala (con 3 columnas: col1,col2,col3),
#que señala el índice de renglón (col1) y columna (col2) de la matriz R
#correspondiente a los elementos distitnos de nan, así como el valor
#de dicho elemento (col3)
MC=compactar_matriz_rala(M)
print('Expresión compacta de M (MC):\n',MC)
print('Tamaño MC:\n',MC.shape)

Expresión compacta de M (MC):
 [[  0.   2.   2.]
 [  0.   4.   4.]
 [  1.   3.   4.]
 [  1.   4.   4.]
 [  2.   1.   5.]
 [  3.   2.   3.]
 [  4.   4.   4.]
 [  5.   1.   4.]
 [  5.   4.   4.]
 [  6.   0.   4.]
 [  8.   3.   1.]
 [ 11.   2.   5.]
 [ 11.   3.   1.]
 [ 12.   1.   5.]
 [ 12.   2.   1.]
 [ 13.   0.   2.]
 [ 14.   0.   1.]
 [ 14.   4.   1.]
 [ 15.   3.   3.]
 [ 15.   4.   5.]
 [ 16.   1.   3.]
 [ 16.   4.   3.]
 [ 18.   1.   5.]
 [ 18.   2.   4.]
 [ 19.   0.   1.]]
Tamaño MC:
 (25, 3)


In [17]:
#Entre las funciones de la librería libmf programado en python
#existe una para generar matrices ralas compactas, sin embargo
#se observa que puede arrojar más de un valor para un mismo
#elemento de la matriz rala, vease el renglón [0,1]
#tiene 3 valores asignados (0.166,0.217,0.556).

#El 1er parámetro indica que se seleccionará un valor
#entre 0 y 19 para la 1er columna (que representa el índice
#de los renglones de la matriz rala que expresa); el 2do
#parámetro indica que se seleccionara al azar un valor entre
#0 y 4 (que representa el índice de las columnas de la matriz
#rala que expresa); el 3er parámetro indica el número de elementos
#a generar 

#Fijaremos una semilla para reproducibilidad
np.random.seed(102)
MC=mf.generate_test_data(20, 5, 25)
print('Matriz generada con mf.generate_test_data(20,5,25):\n')
for i in range(MC.shape[0]):
    if MC[i,0]==0:
        print('\033[1;30;41m',MC[i,:])
    else:
        print('\033[0;0;0m',MC[i,:])
print('Tamaño MC:\n',MC.shape)

Matriz generada con mf.generate_test_data(20,5,25):

 [ 0.     1.     0.166]
 [ 19.      4.      0.306]
 [ 14.      3.      0.147]
 [ 18.      4.      0.223]
 [ 18.     1.     0.54]
 [ 15.      3.      0.749]
 [ 2.     2.     0.428]
 [ 11.      3.      0.105]
 [ 9.     3.     0.753]
 [ 8.     5.     0.598]
 [ 13.      0.      0.487]
 [ 8.     4.     0.975]
 [ 9.     2.     0.802]
 [ 15.      3.      0.333]
 [ 7.     5.     0.549]
 [ 4.     5.     0.312]
 [ 16.      4.      0.062]
 [ 6.     1.     0.903]
 [ 18.      0.      0.861]
 [ 3.    3.    0.63]
 [ 0.     1.     0.217]
 [ 0.     1.     0.556]
 [ 16.      5.      0.109]
 [ 6.     5.     0.551]
 [ 7.     0.     0.581]
Tamaño MC:
 (25, 3)


In [18]:
#El presente trabajo plantea trabajar con bases de películas donde
#los ínidices i de renglones representan usuarios, los j de columnas
#las películas, y el valor de elemento de la matriz la calificación
#asignada a la película j por el usuario i
#Por lo anterior, para crear matrices compactas ralas usaremos funciones propias
#donde un usuario no califique más de una vez una misma película (lo cual 
#sucedería si se genera la matriz de calificaciones con la función de la
#paquetería libmf). La calificación será un no. aleatorio entre 1 y 5
#generada con una distribución uniforme discreta
MC=crear_matriz_rala_compacta(20,5,0.25)
print('Expresión compacta de M (MC) ordenada por renglon:\n',MC[np.argsort(MC[:,0])])
print('Tamaño MC:\n',MC.shape)

Expresión compacta de M (MC) ordenada por renglon:
 [[ 0  4  3]
 [ 1  0  5]
 [ 1  3  1]
 [ 2  2  5]
 [ 3  3  4]
 [ 4  1  4]
 [ 5  3  4]
 [ 5  2  4]
 [ 5  1  2]
 [ 7  1  5]
 [ 9  4  3]
 [10  4  3]
 [11  2  2]
 [11  0  1]
 [14  1  5]
 [15  4  2]
 [15  2  1]
 [15  3  3]
 [16  4  4]
 [16  0  2]
 [17  4  3]
 [17  3  1]
 [18  3  4]
 [19  2  1]
 [19  0  5]]
Tamaño MC:
 (25, 3)


In [19]:
#Se muestra una forma de guardar la matriz compacta en un txt
#que podría ser utilizado directamente con las funciones
#de libmf a través de Shell Commands ("!")
np.savetxt('MC.txt', MC, delimiter=' ', fmt="%.0f")
f = open('MC.txt', 'r')
file_contents = f.read()
print (file_contents)
f.close()

16 0 2
9 4 3
18 3 4
19 0 5
17 3 1
16 4 4
3 3 4
15 2 1
1 0 5
15 4 2
14 1 5
5 1 2
4 1 4
15 3 3
19 2 1
1 3 1
17 4 3
7 1 5
2 2 5
5 2 4
5 3 4
10 4 3
11 2 2
0 4 3
11 0 1



# Calificaciones promedio por usuario y/o película

In [20]:
#Para plantear un "modelo base de referencia", se calculan los medias
#por renglones (mean_ren) y por columnas (mean_col), así como 
#una media total (mean_tot), luego se sustituyen los valores de 
#la matriz (x_ij) por: mean_ren + mean_col - mean_tot
#La función modelo_base_ref regresa los promedios por renglón,
#columna y total. Con estos promedios se puede construir un modelo
#para heterogenizar el uso de la escala (calificaciones)

promedios=modelo_base_ref(MC)
display(promedios)
#Podemos convertir a numpy array para realizar operaciones con
#variables de este tipo
print('\n promedios como numpy array:') 
promedios=promedios.to_numpy()
print(promedios)
#El modelo base de ref quedaría de la siguiente forma:
print('\n El modelo base de referencia es mean_ren + mean_col - mean_tot:') 
#Una media total (mean_tot):')
promedios[:,0]+promedios[:,1]-promedios[:,2]

,row,col,mean_score_by_row,mean_score_by_col,total_score_mean
row,,,,,
0,0,4,3.000000,3.000000,3.08
1,1,0,3.000000,3.250000,3.08
1,1,3,3.000000,2.833333,3.08
2,2,2,5.000000,2.600000,3.08
3,3,3,4.000000,2.833333,3.08
4,4,1,4.000000,4.000000,3.08
5,5,1,3.333333,4.000000,3.08
5,5,2,3.333333,2.600000,3.08
5,5,3,3.333333,2.833333,3.08



 promedios como numpy array:
[[  0.      4.      3.      3.      3.08 ]
 [  1.      0.      3.      3.25    3.08 ]
 [  1.      3.      3.      2.833   3.08 ]
 [  2.      2.      5.      2.6     3.08 ]
 [  3.      3.      4.      2.833   3.08 ]
 [  4.      1.      4.      4.      3.08 ]
 [  5.      1.      3.333   4.      3.08 ]
 [  5.      2.      3.333   2.6     3.08 ]
 [  5.      3.      3.333   2.833   3.08 ]
 [  7.      1.      5.      4.      3.08 ]
 [  9.      4.      3.      3.      3.08 ]
 [ 10.      4.      3.      3.      3.08 ]
 [ 11.      0.      1.5     3.25    3.08 ]
 [ 11.      2.      1.5     2.6     3.08 ]
 [ 14.      1.      5.      4.      3.08 ]
 [ 15.      2.      2.      2.6     3.08 ]
 [ 15.      3.      2.      2.833   3.08 ]
 [ 15.      4.      2.      3.      3.08 ]
 [ 16.      0.      3.      3.25    3.08 ]
 [ 16.      4.      3.      3.      3.08 ]
 [ 17.      3.      2.      2.833   3.08 ]
 [ 17.      4.      2.      3.      3.08 ]
 [ 18.      3.      4.  

array([ 1.   , -2.   ,  1.   , -1.   ,  2.   ,  1.   ,  2.667,  3.667,
        4.667,  3.   , 10.   , 11.   ,  9.5  , 11.5  , 10.   , 15.   ,
       16.   , 17.   , 13.   , 17.   , 18.   , 19.   , 17.   , 16.   ,
       18.   ])

In [21]:
#En el camino para construir modelos adecuados puede ser de interés
#o utilidad identificar los usuarios que tiene más películas calificadas
#o las películas con más calificaciones, más precisamente, que tengan
#más de "n" calificaciones.

#Usuarios que calificaron más de 2 películas
display(filtrar(MC,'row',2))
#Películas que fueron calificadas por más de 2 usuarios
display(filtrar(MC,'col',2))

array([[ 5,  1,  2],
       [ 5,  2,  4],
       [ 5,  3,  4],
       [15,  2,  1],
       [15,  4,  2],
       [15,  3,  3]])

array([[16,  0,  2],
       [19,  0,  5],
       [ 1,  0,  5],
       [11,  0,  1],
       [14,  1,  5],
       [ 5,  1,  2],
       [ 4,  1,  4],
       [ 7,  1,  5],
       [15,  2,  1],
       [19,  2,  1],
       [ 2,  2,  5],
       [ 5,  2,  4],
       [11,  2,  2],
       [18,  3,  4],
       [17,  3,  1],
       [ 3,  3,  4],
       [15,  3,  3],
       [ 1,  3,  1],
       [ 5,  3,  4],
       [ 9,  4,  3],
       [16,  4,  4],
       [15,  4,  2],
       [17,  4,  3],
       [10,  4,  3],
       [ 0,  4,  3]])

# Librería libmf python

In [22]:
#la librería libmf-python programada en python implementa algunas de las funciones
#de libmf. A continuación se corre un demo que exhibe el uso de estas funciones
%cd /home/miuser/python-libmf/tests
!pwd
!python3 mf_tests.py

/home/miuser/python-libmf/tests
/home/miuser/python-libmf/tests
/home/miuser/.local/lib/python3.6/site-packages/libmf/__init__.py
Using file found in /home/miuser/.local/lib/python3.6/site-packages:
/home/miuser/.local/lib/python3.6/site-packages/libmf.cpython-36m-x86_64-linux-gnu.so
iter      tr_rmse          obj
   0       0.5105   3.2420e+02
   1       0.4821   3.0157e+02
   2       0.4340   2.6363e+02
   3       0.3761   2.1923e+02
   4       0.3317   1.8722e+02
   5       0.3064   1.6758e+02
   6       0.2923   1.5558e+02
   7       0.2880   1.5046e+02
   8       0.2836   1.4531e+02
   9       0.2817   1.4216e+02
  10       0.2802   1.3963e+02
  11       0.2789   1.3738e+02
  12       0.2794   1.3651e+02
  13       0.2786   1.3509e+02
  14       0.2788   1.3438e+02
  15       0.2785   1.3355e+02
  16       0.2780   1.3269e+02
  17       0.2785   1.3261e+02
  18       0.2776   1.3176e+02
  19       0.2773   1.3132e+02
testing cross val method
/home/miuser/.local/lib/python3.6/site-

In [23]:
#Esta paquetería tiene unas clases definidas y métodos con los 
#cuales corre algunos de los algoritmos

#Uno de los métodos utilizados para obtener un modelo de recomendación
#es encontrar 2 matrices cuya multiplicación proporciona la estimación
#de calificación. Ambas matrices contienen los llamados "factores latentes"
#que se les puede dar una interpretación de características/gustos
#Esta factorización puede quedar como P*Q_transpuesta

mf_engine=mf.MF()
type(mf_engine)

libmf.mf.MF

In [24]:
#fit method
mf_engine.fit(MC)
mf_engine

In [25]:
#testing factor-p
P=mf_engine.p_factors()
print('Matriz p:\n',P)
print('Tamaño p:\n',P.shape)

Matriz p:
 [[ 0.517  0.679  0.252  0.333  0.621 -0.007 -0.022  0.312]
 [-0.005  0.731  0.443  0.522  0.478  0.327  0.965  1.059]
 [ 0.849  0.98   1.052  1.073  0.294  0.736  0.617  0.471]
 [-0.134  1.26   0.244  0.715 -0.024 -0.396  0.347  1.177]
 [ 2.285  0.662  1.168  1.204  2.296  1.435  1.441  0.591]
 [ 0.827  0.226  0.415  0.919  0.841  0.892  0.828  0.813]
 [   nan    nan    nan    nan    nan    nan    nan    nan]
 [   nan    nan    nan    nan    nan    nan    nan    nan]
 [   nan    nan    nan    nan    nan    nan    nan    nan]
 [   nan    nan    nan    nan    nan    nan    nan    nan]
 [-0.014 -0.014  0.44   0.353  0.313  0.063 -0.143  0.389]
 [ 0.844  1.046  0.154  0.811  0.548  0.791  0.966  0.87 ]
 [   nan    nan    nan    nan    nan    nan    nan    nan]
 [   nan    nan    nan    nan    nan    nan    nan    nan]
 [   nan    nan    nan    nan    nan    nan    nan    nan]
 [ 0.828  1.148  0.887  0.163  0.932  0.643  0.686  0.616]
 [ 1.817  1.248  1.465  1.273  0.867  1.363  

In [26]:
#testing factor-q
Q=mf_engine.q_factors()
print('Matriz Q:\n',Q)
print('Tamaño Q:\n',Q.shape)

Matriz Q:
 [[ 0.042  1.124 -0.021  0.476  0.041  0.23   0.39   0.627]
 [ 0.14   0.372  0.155  0.487  0.166  0.67   0.965  0.416]
 [ 0.494  0.174  0.695  0.785  0.431  0.847  0.529  0.497]
 [ 0.063  0.571  0.521  0.32  -0.251  0.672  0.275  0.121]
 [-0.229  0.742  0.339  0.708  0.011  0.175  0.722  1.264]
 [ 1.02   1.122  0.615  1.276  0.333  0.147  0.372  0.757]
 [   nan    nan    nan    nan    nan    nan    nan    nan]
 [   nan    nan    nan    nan    nan    nan    nan    nan]
 [   nan    nan    nan    nan    nan    nan    nan    nan]
 [ 1.818  1.403  1.519  1.41   0.835  1.302  1.953  1.519]
 [   nan    nan    nan    nan    nan    nan    nan    nan]
 [ 0.109  0.427  0.488 -0.055  0.314  0.349  0.757  0.459]
 [   nan    nan    nan    nan    nan    nan    nan    nan]
 [   nan    nan    nan    nan    nan    nan    nan    nan]
 [ 1.072  1.144  0.566  0.28   0.669  0.648  0.722  0.867]
 [ 2.222  0.572  1.206  1.295  2.183  1.723  1.45   0.474]
 [   nan    nan    nan    nan    nan    nan  

In [27]:
#Se exhibe la multiplicación de matrices P y Q
R_est=P@np.transpose(Q)
print('Matriz p:\n',R_est)
print('Tamaño p:\n',R_est.shape)

Matriz p:
 [[  1.149   0.734   1.215   0.529   1.091   2.303     nan     nan     nan
    3.687     nan   0.771     nan     nan   2.233   3.734     nan   3.415]
 [  2.195   2.265   2.361   1.309   3.161   3.121     nan     nan     nan
    6.743     nan   1.98      nan     nan   3.372   5.125     nan   6.636]
 [  2.343   2.503   3.474   2.153   2.823   4.775     nan     nan     nan
    9.155     nan   1.998     nan     nan   4.454   8.134     nan   8.278]
 [  2.526   1.391   1.304   1.044   3.223   3.292     nan     nan     nan
    4.83      nan   1.26      nan     nan   2.632   1.966     nan   4.339]
 [  2.746   4.314   6.261   2.373   3.28    7.287     nan     nan     nan
   16.053     nan   3.62      nan     nan   8.223  18.28      nan  15.751]
 [  1.789   2.586   3.417   1.406   2.56    3.859     nan     nan     nan
    8.462     nan   1.914     nan     nan   4.079   8.616     nan   8.564]
 [    nan     nan     nan     nan     nan     nan     nan     nan     nan
      nan     nan    

In [28]:
#Validación cruzada
mf_engine.mf_cross_validation(MC)

6.476004031332295

# Obtención de muestra base netflix

In [29]:
#Se guarda una variable con una muestra de las base de netflix ("movies_title_fix.csv")
#A continuación se presentan las características de las películas, con el objeto
#de conocer un poco más los datos utilizados (pero esta no será la base usada para el modelo)
pelis_nombres=pd.read_csv('movies_title_fix.csv',names=["peli_id", "año", "nombre"])
#Para ver cómo se encuentran los datos
pelis_nombres.head(10) 

,peli_id,año,nombre
0,1,2003.0,Dinosaur Planet
1,2,2004.0,Isle of Man TT 2004 Review
2,3,1997.0,Character
3,4,1994.0,Paula Abdul's Get Up & Dance
4,5,2004.0,The Rise and Fall of ECW
5,6,1997.0,Sick
6,7,1992.0,8 Man
7,8,2004.0,What the #$*! Do We Know!?
8,9,1991.0,Class of Nuke 'Em High 2
9,10,2001.0,Fighter


In [30]:
#La base tiene peli_id, usuario_id_orig, calif, fecha y usuario_id. Se trabajará solo
#con las columnas de usuario, película y calificación
df_muestra_netflix=pd.read_csv('dat_muestra_nflix.csv')
df_muestra_netflix_libmf=df_muestra_netflix[['usuario_id', 'peli_id', 'calif']]
df_muestra_netflix_libmf.shape

(20968941, 3)

In [31]:
#Se genera una base de validación que contendrá el 20% de los datos
#y una base de entrenamiento que tendrá el restante 80%
random.seed(28882)
data_valida, data_entrena=muestra_val_ent(df_muestra_netflix_libmf,0.2)

#Se pueden guardar las bases como un .csv dentro del directorio donde está este notebook
data_valida.to_csv(r'data_valida.csv', index = False)    
data_entrena.to_csv(r'data_entrena.csv', index = False)    

# #Base de todos los usuarios
# usuarios=pd.DataFrame(df_muestra_netflix_libmf['usuario_id'].unique(), columns=['usuario_id'])
# #Muestreo de usuarios
# valida_usuarios=usuarios.sample(frac=0.2)  

# #Base de todas las películas
# peliculas=pd.DataFrame(df_muestra_netflix_libmf['peli_id'].unique(), columns=['peli_id'])
# #Muestreo de películas
# valida_pelis=peliculas.sample(frac=0.2)

In [32]:
# #df_valida tiene las columnas usuario_id, peli_id y calif, pero
# #sólo incluye los usuarios muestreados en valida_usuarios
# df_valida=pd.merge(df_muestra_netflix_libmf,valida_usuarios, on ='usuario_id', how='inner')
# #data_valida filtra de la base df_valida los pelis_id muestreas
# #en valida_pelis
# data_valida=pd.merge(df_valida,valida_pelis, on ='peli_id', how='inner')
# #Guardamos data_valida como un .csv dentro del directorio donde está este notebook
# data_valida.to_csv(r'muestra_valida.csv', index = False)
# left= pd.merge(df_muestra_netflix_libmf, data_valida, how='left',on=['usuario_id', 'peli_id'])
# data_entrena=left[left.isnull().any(axis=1)][['usuario_id', 'peli_id','calif_x']]

In [33]:
display(data_valida.head(5))
data_valida.shape

,usuario_id,peli_id,calif
0,7,28,3
1,12,28,2
2,18,28,3
3,34,28,2
4,42,28,5


(832603, 3)

In [34]:
display(data_entrena.head(5))
data_entrena.shape

,usuario_id,peli_id,calif_x
0,1,1,3
1,2,1,3
2,3,1,4
3,4,1,4
4,5,1,4


(20136338, 3)

In [35]:
#Podemos cambiar el directorio para que se guarde la base
#dentro del mismo donde están las funciones de libmf y
#se guarda con la extensión .tr.txt porque es la requerida
#por la paquetería de libmf
%cd /home/miuser/libmf/demo
np.savetxt('data_valida.te.txt', data_valida.to_numpy(), delimiter=' ', fmt="%.0f")
np.savetxt('data_entrena.tr.txt', data_entrena.to_numpy(), delimiter=' ', fmt="%.0f")

/home/miuser/libmf/demo


In [36]:
#La base de df_muestra_valida_libmf ha quedado dividida por
#dos dataframes excluyentes: data_valida y data_entrena
print("La suma de elementos de data_valida + data_entrena:",data_valida.shape[0] + data_entrena.shape[0])
print("El total de elementos de df_muestra_netflix_libmf es:",df_muestra_netflix_libmf.shape[0])

La suma de elementos de data_valida + data_entrena: 20968941
El total de elementos de df_muestra_netflix_libmf es: 20968941


In [37]:
modelo_base_ref(data_valida.to_numpy())

,row,col,mean_score_by_row,mean_score_by_col,total_score_mean
row,,,,,
7,7,28,4.186813,3.797207,3.589491
7,7,499,4.186813,3.047244,3.589491
7,7,1220,4.186813,4.011984,3.589491
7,7,1387,4.186813,2.545455,3.589491
7,7,1558,4.186813,3.008451,3.589491
...,...,...,...,...,...
99830,99830,13402,4.000000,3.878476,3.589491
99859,99859,14215,1.000000,2.831273,3.589491
99875,99875,14526,5.000000,3.675926,3.589491


# Librería libmf

In [38]:
#Probar correr el demo que viene en el la librería libmf
%cd /home/miuser/libmf/demo
!pwd
!bash demo.sh

/home/miuser/libmf/demo
/home/miuser/libmf/demo
--------------------------------
Real-valued matrix factorization
--------------------------------
iter      tr_rmse      va_rmse          obj
   0       2.4804       1.3263   3.2505e+04
   1       1.1370       1.0718   8.1881e+03
   2       0.8945       1.0377   5.7280e+03
   3       0.8020       1.0173   4.9459e+03
   4       0.7456       1.0091   4.4984e+03
   5       0.6996       1.0113   4.1878e+03
   6       0.6556       1.0048   3.9055e+03
   7       0.6093       1.0056   3.6412e+03
   8       0.5625       1.0072   3.4007e+03
   9       0.5124       1.0048   3.1607e+03
MAE = 0.7801
---------------------------
binary matrix factorization
---------------------------
iter   tr_logloss   va_logloss          obj
   0       0.6888       0.8163   3.4252e+03
   1       0.6957       0.8171   3.4580e+03
   2       0.6816       0.8178   3.3886e+03
   3       0.6666       0.8125   3.3169e+03
   4       0.6506       0.8011   3.2421e+03
   5    

In [39]:
%cd /home/miuser/libmf/demo

/home/miuser/libmf/demo


In [40]:
#train a model using the default parameters
!/home/miuser/libmf/mf-train real_matrix.tr.txt model

iter      tr_rmse          obj
   0       1.9774   2.2897e+04
   1       0.9920   8.1689e+03
   2       0.8694   6.9750e+03
   3       0.8157   6.4784e+03
   4       0.7858   6.1998e+03
   5       0.7651   5.9973e+03
   6       0.7521   5.8683e+03
   7       0.7396   5.7746e+03
   8       0.7320   5.6996e+03
   9       0.7196   5.5783e+03
  10       0.7167   5.5489e+03
  11       0.7101   5.5005e+03
  12       0.7072   5.4800e+03
  13       0.7003   5.4107e+03
  14       0.6934   5.3499e+03
  15       0.6865   5.3164e+03
  16       0.6876   5.3167e+03
  17       0.6814   5.2760e+03
  18       0.6778   5.2405e+03
  19       0.6732   5.2029e+03


In [41]:
#train a model with the following regularization coefficients:
#coefficient of L1-norm regularization on P = 0.05
#coefficient of L1-norm regularization on Q = 0.05
#coefficient of L2-norm regularization on P = 0.01
#coefficient of L2-norm regularization on Q = 0.01
!/home/miuser/libmf/mf-train -l1 0.05 -l2 0.01 real_matrix.tr.txt model

iter      tr_rmse          obj
   0       1.9895   2.2903e+04
   1       0.9930   8.0382e+03
   2       0.8601   6.8028e+03
   3       0.8046   6.3472e+03
   4       0.7703   6.0623e+03
   5       0.7425   5.8543e+03
   6       0.7222   5.6903e+03
   7       0.7088   5.5888e+03
   8       0.6937   5.4709e+03
   9       0.6781   5.3715e+03
  10       0.6662   5.2890e+03
  11       0.6506   5.1755e+03
  12       0.6371   5.0819e+03
  13       0.6246   5.0102e+03
  14       0.6123   4.9264e+03
  15       0.5985   4.8426e+03
  16       0.5847   4.7569e+03
  17       0.5748   4.6928e+03
  18       0.5617   4.6187e+03
  19       0.5511   4.5604e+03


In [42]:
#train a model with the following regularization coefficients:
#coefficient of L1-norm regularization on P = 0.05
#coefficient of L1-norm regularization on Q = 0
#coefficient of L2-norm regularization on P = 0.01
#coefficient of L2-norm regularization on Q = 0.03
!/home/miuser/libmf/mf-train -l1 0.015,0 -l2 0.01,0.005 real_matrix.tr.txt model

iter      tr_rmse          obj
   0       1.9589   1.9665e+04
   1       0.9769   5.2544e+03
   2       0.8479   4.0808e+03
   3       0.7934   3.6375e+03
   4       0.7633   3.4040e+03
   5       0.7379   3.2165e+03
   6       0.7228   3.1081e+03
   7       0.7074   2.9989e+03
   8       0.6924   2.8970e+03
   9       0.6789   2.8042e+03
  10       0.6652   2.7150e+03
  11       0.6514   2.6260e+03
  12       0.6382   2.5434e+03
  13       0.6238   2.4547e+03
  14       0.6107   2.3764e+03
  15       0.5956   2.2874e+03
  16       0.5829   2.2149e+03
  17       0.5706   2.1459e+03
  18       0.5566   2.0692e+03
  19       0.5435   1.9992e+03


In [43]:
#train a model with the following regularization coefficients:
#coefficient of L1-norm regularization on P = 0.05
#coefficient of L1-norm regularization on Q = 0
#coefficient of L2-norm regularization on P = 0.01
#coefficient of L2-norm regularization on Q = 0.03
!/home/miuser/libmf/mf-train -l1 0.015,0 -l2 0.01,0.005 real_matrix.tr.txt model

iter      tr_rmse          obj
   0       1.9589   1.9665e+04
   1       0.9769   5.2544e+03
   2       0.8479   4.0808e+03
   3       0.7951   3.6525e+03
   4       0.7634   3.4054e+03
   5       0.7413   3.2408e+03
   6       0.7231   3.1106e+03
   7       0.7047   2.9793e+03
   8       0.6928   2.8977e+03
   9       0.6730   2.7658e+03
  10       0.6650   2.7141e+03
  11       0.6494   2.6133e+03
  12       0.6373   2.5373e+03
  13       0.6244   2.4578e+03
  14       0.6106   2.3754e+03
  15       0.5952   2.2843e+03
  16       0.5837   2.2192e+03
  17       0.5697   2.1414e+03
  18       0.5573   2.0732e+03
  19       0.5441   2.0025e+03


In [44]:
#train a BMF model using logarithmic loss and the following parameters:
#coefficient of L1-norm regularization on P = 0
#coefficient of L1-norm regularization on Q = 0.01
#latent factors = 100
#iterations = 30
#learning rate = 0.02
#threads = 4
!/home/miuser/libmf/mf-train -f 5 -l1 0,0.02 -k 100 -t 30 -r 0.02 -s 4 binary_matrix.tr.txt model

iter   tr_logloss          obj
   0       0.6573   3.8976e+03
   1       0.6948   4.0099e+03
   2       0.6915   3.9364e+03
   3       0.6891   3.8791e+03
   4       0.6872   3.8318e+03
   5       0.6859   3.7937e+03
   6       0.6848   3.7616e+03
   7       0.6840   3.7351e+03
   8       0.6834   3.7128e+03
   9       0.6828   3.6929e+03
  10       0.6824   3.6765e+03
  11       0.6820   3.6620e+03
  12       0.6817   3.6491e+03
  13       0.6815   3.6380e+03
  14       0.6813   3.6276e+03
  15       0.6811   3.6187e+03
  16       0.6810   3.6106e+03
  17       0.6808   3.6031e+03
  18       0.6807   3.5962e+03
  19       0.6806   3.5904e+03
  20       0.6805   3.5848e+03
  21       0.6804   3.5796e+03
  22       0.6804   3.5749e+03
  23       0.6803   3.5703e+03
  24       0.6803   3.5663e+03
  25       0.6802   3.5622e+03
  26       0.6802   3.5587e+03
  27       0.6801   3.5554e+03
  28       0.6801   3.5522e+03
  29       0.6801   3.5493e+03


In [45]:
#use real_matrix.te.txt for hold-out validation
!/home/miuser/libmf/mf-train -p real_matrix.te.txt real_matrix.tr.txt model

iter      tr_rmse      va_rmse          obj
   0       1.9775       1.1537   2.2900e+04
   1       0.9888       1.0483   8.1391e+03
   2       0.8672       1.0284   6.9628e+03
   3       0.8163       1.0161   6.4664e+03
   4       0.7837       1.0120   6.1815e+03
   5       0.7667       1.0084   6.0290e+03
   6       0.7522       1.0065   5.8825e+03
   7       0.7404       1.0132   5.7769e+03
   8       0.7321       1.0027   5.7059e+03
   9       0.7237       1.0045   5.6217e+03
  10       0.7169       1.0056   5.5626e+03
  11       0.7103       1.0033   5.5057e+03
  12       0.7042       1.0042   5.4677e+03
  13       0.7009       1.0042   5.4101e+03
  14       0.6950       1.0082   5.3894e+03
  15       0.6878       1.0055   5.3017e+03
  16       0.6886       1.0047   5.3340e+03
  17       0.6820       1.0049   5.2721e+03
  18       0.6763       1.0071   5.2148e+03
  19       0.6720       1.0082   5.1932e+03


In [46]:
#do five fold cross validation
!/home/miuser/libmf/mf-train -v 5 real_matrix.tr.txt

fold      rmse
   0    1.4038
   1    1.4244
   2    1.5077
   3    1.5475
   4    1.5100
 avg    1.4787


In [47]:
#do non-negative matrix factorization with generalized KL-divergence
!/home/miuser/libmf/mf-train -f 2 --nmf real_matrix.tr.txt

iter       tr_gkl          obj
   0       0.9901   8.0001e+03
   1       0.2301   3.9768e+03
   2       0.1756   3.6750e+03
   3       0.1564   3.5405e+03
   4       0.1453   3.4320e+03
   5       0.1382   3.3839e+03
   6       0.1322   3.3386e+03
   7       0.1278   3.3060e+03
   8       0.1234   3.2575e+03
   9       0.1214   3.2492e+03
  10       0.1180   3.2155e+03
  11       0.1162   3.1949e+03
  12       0.1145   3.1802e+03
  13       0.1124   3.1601e+03
  14       0.1117   3.1581e+03
  15       0.1096   3.1399e+03
  16       0.1086   3.1186e+03
  17       0.1083   3.1339e+03
  18       0.1069   3.1187e+03
  19       0.1058   3.1196e+03


In [48]:
#do not print message to screen
!/home/miuser/libmf/mf-train --quiet real_matrix.tr.txt

In [49]:
#do disk-level training
!/home/miuser/libmf/mf-train --disk real_matrix.tr.txt

iter      tr_rmse          obj
   0       1.7077   1.7882e+04
   1       1.1646   9.9921e+03
   2       0.8696   6.9916e+03
   3       0.8046   6.4085e+03
   4       0.7730   6.0870e+03
   5       0.7558   5.9400e+03
   6       0.7463   5.8392e+03
   7       0.7351   5.7426e+03
   8       0.7240   5.6205e+03
   9       0.7198   5.5841e+03
  10       0.7112   5.5185e+03
  11       0.7067   5.4698e+03
  12       0.7020   5.4230e+03
  13       0.6955   5.3818e+03
  14       0.6896   5.3262e+03
  15       0.6859   5.3009e+03
  16       0.6776   5.2387e+03
  17       0.6733   5.2200e+03
  18       0.6705   5.1927e+03
  19       0.6665   5.1421e+03


In [50]:
#do prediction
!/home/miuser/libmf/mf-predict real_matrix.te.txt model output

RMSE = 1.0082


In [51]:
#do prediction and output MAE
!/home/miuser/libmf/mf-predict -e 1 real_matrix.te.txt model output

MAE = 0.7858


In [52]:
#Pruebese los diferentes modelos ejemplo con data_entrena y data_valida
#En general estos algoritmos corren muy rápido
!/home/miuser/libmf/mf-train data_entrena.tr.txt model

iter      tr_rmse          obj
   0       0.9814   3.0187e+07
   1       0.9347   2.8346e+07
   2       0.9301   2.8165e+07
   3       0.9256   2.8044e+07
   4       0.9190   2.7872e+07
   5       0.9112   2.7666e+07
   6       0.9045   2.7490e+07
   7       0.8998   2.7364e+07
   8       0.8966   2.7275e+07
   9       0.8944   2.7215e+07
  10       0.8928   2.7178e+07
  11       0.8917   2.7142e+07
  12       0.8907   2.7116e+07
  13       0.8900   2.7101e+07
  14       0.8893   2.7085e+07
  15       0.8888   2.7068e+07
  16       0.8883   2.7056e+07
  17       0.8879   2.7048e+07
  18       0.8874   2.7039e+07
  19       0.8870   2.7023e+07


In [53]:
######################## BASURA / CÓDIGO DESECHADO #######################

In [54]:
# #testing predict method
# pred_data = mf.generate_test_data(9, 4, 5, indices_only=True)
# predictions = mf_engine.predict(pred_data)


In [55]:
# #$ python tests/mf_tests.py
# %run 'mf_tests.py'


In [56]:
# import csv
# with open('dat_muestra_nflix.csv', newline='') as csvfile:
#     spamreader = csv.reader(csvfile, delimiter=' ', quotechar='|')

In [57]:
# spamreader[1]

In [58]:
# data = pd.read_csv("dat_muestra_nflix.csv", sep=' ')

In [59]:
# def centralizar_valores(M):
#     n=M.shape[0]
    
 
    
#     #Se hace una copia de la matriz M
#         M_copy=copy.copy(M)
#         #Se agrega una 4ta columna
#         M_copy=np.append(M, np.array(n*[np.nan]).reshape(n,1), axis=1)
#         #Se ordena la matriz
#         M_copy=M_copy[np.argsort(M_copy[:,0])]
#         #k1 irá marcando el renglón donde van cambiando los valores de la 1era columna de M
        
        
#         valores=np.zeros(1)
#         for i in range(1,M_copy.shape[0]):
#             if M_copy[i,2]!=M_copy[i-1,2]:
#                 print('lo que sea')
            
            
#             valores[k1]=np.r_[valores, np.zeros(1)]
#             for j in range(k1,k2+1):
#                 #M_copy[j,4]
#                 print('lo que sea')
# #     else:
# #         calif_cent=np.nanmean(M, axis=1).reshape(n,1)
# #         return M-calif_cent

In [60]:
# help(mf)

In [61]:
# print([getattr(mf, a)
# for a in dir(mf)
#   if isinstance(getattr(mf, a), types.FunctionType)])

In [62]:
#help(mf.generate_test_data)

In [63]:
#dir(mf_engine)

In [64]:
# #Uso de función generate_test_data
# #para generar un dataset de prueba con mil renglones, 
# #cuyos elementos de la 1era, columna van de 0 a 100, 
# #y los de la segunda columna de 0 a 50, la tercera 
# #columna son numeros aleatorios 
# test_data = mf.generate_test_data(9, 4, 20)
# print('Matriz de prueba:\n',test_data)
# print('Tamaño de la matriz:\n',test_data.shape)

In [65]:
# def myFun(arg1, *argv): 
#     print(type(argv))
#     if len(argv)==0:
#         print('no hay argumentos variables')
#     print ("First argument :", arg1) 
#     for arg in argv: 
#         print("Next argument through *argv :", arg)
# #myFun(['hola','tu','chales'])
# myFun(*[3])

In [66]:
# nan_or_value_vec_one_arg=np.vectorize(nan_or_value)
# nan_or_value_vec_mult_args=np.vectorize(nan_or_value, otypes=[object])
# def nan_or_value_vec(M,f,*args):
#     if len(args)==1:
#         print('usase la función constante?')
#         return nan_or_value_vec_one_arg(M,f,args)
#     else:
#         return nan_or_value_vec_mult_args(M,f,*args)

In [67]:
# print('Numero de elementos distintos de cero:\n',np.count_nonzero(R))

In [68]:
#display(pd.DataFrame(MC).sort_values(by=['row','col'], ascending=True))

In [69]:
#f = open('data_valida.txt', 'r')
#file_contents = f.read()
#print (file_contents)
#f.close()

In [70]:
#np.set_printoptions(sign=' ',formatter={'float': lambda x: "{0:0.3f}".format(x)})